In [209]:
import numpy as np
import qutip as qt
from scipy.linalg import logm, expm
from qiskit.quantum_info import Operator, state_fidelity
from qiskit import QuantumCircuit
from qiskit_aer import StatevectorSimulator
from qiskit import Aer
from qiskit.circuit.library import QFT

from op_fourier_trafo import *
from tools.classical import *
from tools.quantum import *
from jump_ops import sigmam_LCU

In [210]:
num_qubits = 3
eps = 0
bohr_bound = 0

hamiltonian = find_ideal_heisenberg(num_qubits, bohr_bound, eps, signed=False, for_oft=False)

Original spectrum:  [-4.   -3.46 -2.83 -2.    2.    2.83  3.46  4.  ]
Ideal spectrum:  [-0.    0.07  0.15  0.25  0.75  0.85  0.93  1.  ]
Nonrescaled coefficients:  [1. 1. 1. 1.]
Rescaled coefficients:  [0.12 0.12 0.12 0.12]


In [211]:
jump_op = sigmam_LCU()
print(jump_op)
left_proj = qt.tensor([qt.basis(2, 0).dag(), qt.qeye(2**num_qubits)]).full()  #! Dagger doesnt change order
right_proj = qt.tensor([qt.basis(2, 0), qt.qeye(2**num_qubits)]).full()
np.set_printoptions(precision=2, suppress=True)


                     ┌───┐     ┌───┐           
sys: ────────────────┤ X ├─────┤ Y ├───────────
     ┌─────────┐┌───┐└─┬─┘┌───┐└─┬─┘┌─────────┐
  b: ┤ Unitary ├┤ X ├──■──┤ X ├──■──┤ Unitary ├
     └─────────┘└───┘     └───┘     └─────────┘


In [212]:
qr_sys = QuantumRegister(num_qubits, 'sys')
qr_b = QuantumRegister(1, 'b')
cr_b = ClassicalRegister(1, 'cb')
circ = QuantumCircuit(qr_sys, qr_b, cr_b)

# np.random.seed(666)
randstate_better = np.zeros(2**(num_qubits))
randstate_better[np.random.choice(2**num_qubits, 2**num_qubits // 2, replace=False)] = 1  #! // 2

# randstate_better = np.array([1., 0.])
randstate_better /= np.linalg.norm(randstate_better)

circ.initialize(randstate_better, qr_sys)
circ.barrier()

circ.compose(jump_op, [qr_sys[0], qr_b[0]], inplace=True)
print(circ)
circ_sv = Statevector(circ)

circ2 = QuantumCircuit(qr_sys, qr_b, cr_b)
circ2.compose(jump_op, [qr_sys[0], qr_b[0]], inplace=True)
circ_op = Operator(circ2).data
print(circ_op)

       ┌──────────────────────────────────────┐ ░                 ┌───┐     »
sys_0: ┤0                                     ├─░─────────────────┤ X ├─────»
       │                                      │ ░                 └─┬─┘     »
sys_1: ┤1 Initialize(0.5,0.5,0,0.5,0,0,0.5,0) ├─░───────────────────┼───────»
       │                                      │ ░                   │       »
sys_2: ┤2                                     ├─░───────────────────┼───────»
       └──────────────────────────────────────┘ ░ ┌─────────┐┌───┐  │  ┌───┐»
    b: ─────────────────────────────────────────░─┤ Unitary ├┤ X ├──■──┤ X ├»
                                                ░ └─────────┘└───┘     └───┘»
 cb: 1/═════════════════════════════════════════════════════════════════════»
                                                                            »
«       ┌───┐           
«sys_0: ┤ Y ├───────────
«       └─┬─┘           
«sys_1: ──┼─────────────
«         │             
«sys_2: ──┼──────

In [213]:
sigmam = np.array([[0, 0], [1, 0]])
padded_sigmam = pad_term([qt.Qobj(sigmam)], num_qubits, 0).full()
state_we_want = padded_sigmam @ randstate_better
# state_we_want = sigmam @ randstate_better
state_we_want /= np.linalg.norm(state_we_want)
print(state_we_want)

[0.  +0.j 0.71+0.j 0.  +0.j 0.  +0.j 0.  +0.j 0.  +0.j 0.  +0.j 0.71+0.j]


In [214]:
circ_sv = circ_sv.data
projected_state = left_proj @ circ_sv
print(np.linalg.norm(projected_state))
projected_state /= np.linalg.norm(projected_state)
print(projected_state)
print(state_fidelity(Statevector(projected_state), Statevector(state_we_want), validate=False))

dist_to_what_we_want_with_states = np.linalg.norm(projected_state - state_we_want)
dist_to_what_we_want_with_states

0.7071067811865476
[0.  +0.j 0.71+0.j 0.  +0.j 0.  +0.j 0.  +0.j 0.  +0.j 0.  +0.j 0.71+0.j]
0.9999999999999996


1.1102230246251564e-16

In [215]:
# zero_projector = pad_term([qt.Qobj(np.array([[1, 0], [0, 0]]))], num_qubits + 1, num_qubits).full()
# # zero_projector = np.array([[1, 0], [0, 0]])
# projected_sv = Statevector(zero_projector @ circ_sv.data)
# projected_sys_sv = partial_trace(projected_sv, [num_qubits])

In [216]:
# padded_sigmam = pad_term([qt.Qobj(sigmam)], num_qubits, 1).full()
# state_we_want = padded_sigmam @ randstate_better
# state_we_want = DensityMatrix(state_we_want)

# dist_to_what_we_want = np.linalg.norm(projected_sys_sv - state_we_want)
# print(dist_to_what_we_want)
# state_fidelity(projected_sys_sv, state_we_want, validate=False)

### Test for even more qregs

In [242]:
num_energy_bits = 4
num_qubtis = 3
energy_randstate = np.zeros(2**num_energy_bits)
energy_randstate[np.random.choice(2**num_energy_bits, 2**num_energy_bits // 2, replace=False)] = 1
energy_randstate /= np.linalg.norm(energy_randstate)

initial_energy_sys_state = qt.tensor([qt.Qobj(randstate_better), qt.Qobj(energy_randstate)]).full()
initial_energy_sys_state /= np.linalg.norm(initial_energy_sys_state)

#* Circ
qr_sys = QuantumRegister(num_qubtis, 'sys')
qr_b = QuantumRegister(1, 'b')
qr_energy = QuantumRegister(num_energy_bits, 'w')

circ = QuantumCircuit(qr_energy, qr_sys, qr_b)
circ.initialize(randstate_better, qr_sys)
circ.initialize(energy_randstate, qr_energy)
circ.barrier()

jump_index = 0
circ.compose(jump_op, [qr_sys[jump_index], qr_b[0]], inplace=True)
print(circ)
# # circ_op = Operator(circ).data
circ_sv = Statevector(circ)

#* Project, compare
big_left_proj = qt.tensor([qt.basis(2, 0).dag(), qt.qeye(2**num_qubtis), qt.qeye(2**num_energy_bits)]).full()
projected_state = big_left_proj @ circ_sv.data
projected_state /= np.linalg.norm(projected_state)

padded_sigmam = pad_term([qt.Qobj(sigmam)], num_qubits, jump_index).full()
sys_state_we_want = padded_sigmam @ randstate_better
sys_state_we_want /= np.linalg.norm(sys_state_we_want)
state_we_want = qt.tensor([qt.Qobj(sys_state_we_want), qt.Qobj(energy_randstate)]).full()
state_we_want /= np.linalg.norm(state_we_want)

print(np.linalg.norm(projected_state - state_we_want))


# sys_b_dm = partial_trace(circ_sv, list(range(num_energy_bits)))

# left_proj = qt.tensor([qt.basis(2, 0).dag(), qt.qeye(2**num_qubits)]).full()
# right_proj = qt.tensor([qt.basis(2, 0), qt.qeye(2**num_qubits)]).full()

# projected_state = left_proj @ sys_b_dm.data @ right_proj
# projected_state /= np.trace(projected_state)

# padded_sigmam = pad_term([qt.Qobj(sigmam)], num_qubits, 0).full()
# state_we_want = padded_sigmam @ randstate_better
# state_we_want /= np.linalg.norm(state_we_want)
# dm_we_want = state_we_want @ state_we_want.conj().T

# np.linalg.norm(projected_state - dm_we_want)


       »
  w_0: »
       »
  w_1: »
       »
  w_2: »
       »
  w_3: »
       »
sys_0: »
       »
sys_1: »
       »
sys_2: »
       »
    b: »
       »
«       ┌──────────────────────────────────────────────────────────────────────────────────────────────┐»
«  w_0: ┤0                                                                                             ├»
«       │                                                                                              │»
«  w_1: ┤1                                                                                             ├»
«       │  Initialize(0.35355,0,0,0,0,0.35355,0.35355,0,0.35355,0,0.35355,0.35355,0.35355,0,0,0.35355) │»
«  w_2: ┤2                                                                                             ├»
«       │                                                                                              │»
«  w_3: ┤3                                                                                             ├»